### Doc2vec

In [5]:
!pip install pyMuPDF

  Using cached PyMuPDF-1.21.1-cp310-cp310-win_amd64.whl (11.7 MB)


In [7]:
!pip install nltk

  Using cached nltk-3.8-py3-none-any.whl (1.5 MB)
  Using cached regex-2022.10.31-cp310-cp310-win_amd64.whl (267 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)


### Importing Libraries

In [17]:
import fitz
import os
import re
from nltk.corpus import stopwords
from gensim.models import doc2vec

### Reading Data
---
In this case I have downloaded the pdf and I am reading from them using pyMuPDF library. We may need to tweak this step accoridng to the form of data we need to read.
I, however prefer pdf's mainly for the reason that the uploaded docs are generally in pdf format , and preprocessing is easier to do

In [62]:
doclist = []
i = 1
doc=""
folder = os.listdir('pdffiles')
for file in folder:
    with fitz.open(rf"pdffiles/{file}") as pdf:
        for page in pdf:
            text = page.get_text()
            doc = doc+text
    doclist.append(doc)
    doc=""
    print("file",i," added to doclist")
    i+=1

file 1  added to doclist
file 2  added to doclist
file 3  added to doclist
file 4  added to doclist
file 5  added to doclist
file 6  added to doclist
file 7  added to doclist
file 8  added to doclist
file 9  added to doclist
file 10  added to doclist
file 11  added to doclist
file 12  added to doclist
file 13  added to doclist
file 14  added to doclist
file 15  added to doclist
file 16  added to doclist
file 17  added to doclist
file 18  added to doclist
file 19  added to doclist
file 20  added to doclist


### Text Preprocessing
---
- Remove all the white spaces and extra new line characters
- Remove all the special characters and numbers , keep only a-z, A-Z and spaces
- lower the text and split it and remove stopwords and words with lenght 1.
- return the text as an _array of words_ present in the document

In [63]:
def preprocess(text):
    text = re.sub('[^a-zA-Z ]',' ',text)
    text = re.sub('[^\w\s\']',' ',text)
    text = re.sub('[ \n]+',' ',text)
    text = text.lower().split()     
    text = [word for word in text if (word not in stopwords.words('english') and len(word)>1)] 
    return text
    

In [64]:
corpus=[]
for doc in doclist:
    corpus.append(preprocess(doc))

In [ ]:
print(corpus[1])

### Tagged Documents
---
We have follow a specific format to train and fit the doc2vec model, so we are doing that here

In [66]:
def tagged_document(list_of_list_of_words):
   for i, list_of_words in enumerate(list_of_list_of_words):
      yield doc2vec.TaggedDocument(list_of_words, [i])
data_for_training = list(tagged_document(corpus))

In [ ]:
print(data_for_training)

### Fit, build and train the model
---

We could train better by changing the paramenters

In [65]:
model = doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [67]:
model.build_vocab(data_for_training)

### Calculating the similarity of 2 documents
--- 
we use and inbuilt function which gives the cosine similarity between 2 documents.
_I observed that it ranges from -1 to 1_
may be we could do something like (1+similarity)/2 to map it from -1 to 1 to 0 to 1 but will look into it later.

In [68]:
model.similarity_unseen_docs(corpus[0],corpus[5])

-0.31699306

In [70]:
for i in range(20):
    for j in range(20):
        print("The model similarity between document",i+1,"and document",j+1,"is",(model.similarity_unseen_docs(corpus[i],corpus[j]))*100,"%")

The model similarity between document 1 and document 1 is 100.0 %
The model similarity between document 1 and document 2 is 24.508129060268402 %
The model similarity between document 1 and document 3 is -16.597265005111694 %
The model similarity between document 1 and document 4 is -6.957912445068359 %
The model similarity between document 1 and document 5 is -6.936515122652054 %
The model similarity between document 1 and document 6 is -31.699305772781372 %
The model similarity between document 1 and document 7 is -2.6037603616714478 %
The model similarity between document 1 and document 8 is 12.031739205121994 %
The model similarity between document 1 and document 9 is -6.86919167637825 %
The model similarity between document 1 and document 10 is 11.170253902673721 %
The model similarity between document 1 and document 11 is -14.214906096458435 %
The model similarity between document 1 and document 12 is -0.11169238714501262 %
The model similarity between document 1 and document 13 i

Changing /mapping the range is still pending

Also, using ***universal sentence encoder*** seems to be a better model, but I still need to read more and understand it completely_